In [8]:

import pickle
import psqlparse
from Utility import Encoding as en
from Utility import ParserForIndex as pi
import os

enc = en.encoding_schema()
# path to your tpch_directory/dbgen
work_dir = "../tpch-tool/dbgen"
print(os.path.exists("../tpch-tool/dbgen/queries/"))


True


In [84]:
print(os.environ.copy())

{'ELECTRON_RUN_AS_NODE': '1', 'USER': 'wangtaiyi', 'MallocNanoZone': '0', '__CFBundleIdentifier': 'com.microsoft.VSCode', 'COMMAND_MODE': 'unix2003', 'DISPLAY': '/private/tmp/com.apple.launchd.OKLcKUUO6X/org.macosforge.xquartz:0', 'LOGNAME': 'wangtaiyi', 'PATH': '/Users/wangtaiyi/opt/anaconda3/bin:/Users/wangtaiyi/opt/anaconda3/condabin:/Users/wangtaiyi/opt/anaconda3/bin:/usr/local/texlive/2019/bin/x86_64-darwin:/Users/wangtaiyi/Library/Python/3.8/bin:/Library/Frameworks/Python.framework/Versions/3.8/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/opt/X11/bin:/Library/Apple/usr/bin', 'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.9SI7drDMTv/Listeners', 'SHELL': '/bin/zsh', 'HOME': '/Users/wangtaiyi', '__CF_USER_TEXT_ENCODING': '0x1F5:0x0:0x0', 'TMPDIR': '/var/folders/_l/0clwfzj51wlcmzwq9r9g5jfr0000gn/T/', 'XPC_SERVICE_NAME': '0', 'XPC_FLAGS': '0x0', 'ORIGINAL_XDG_CURRENT_DESKTOP': 'undefined', 'VSCODE_CWD': '/', 'VSCODE_NLS_CONFIG': '{"locale":"en-us","available

In [22]:
import json
import os
import subprocess


class TPCH:
    def __init__(self, query_root, number):
        self.query_root = os.path.abspath(query_root)
        self.dss_query_path = os.path.join(query_root, "queries")
        self.query_env = os.environ.copy()
        self.dbgen_dir = query_root
        self.query_env['DSS_QUERY'] = "/Users/wangtaiyi/Documents/Graduate/Cambridge/Research/RL/Learning_Index_Selection/index_Code/Multi-level_Index/tpch-tool/dbgen/queries"
        # self.query_env['DSS_QUERY'] = "queries"
        self.number = number

    def gen_one_query(self, i):
        # p = subprocess.Popen([os.path.join(".", "qgen"), str(i)], cwd=self.dbgen_dir, env = self.query_env, stdout=subprocess.PIPE)
        p = subprocess.Popen([os.path.join(".","qgen"), str(i)],  cwd=self.dbgen_dir, env = self.query_env,stdout=subprocess.PIPE)
        lines = p.stdout.readlines()
        sql = ""
        for i, line in enumerate(lines):
            sql += str(line, encoding="utf-8")
        p.communicate()
        return sql

    def gen_workloads(self):
        results = list()
        actual = 0
        _seq = 0
        while actual < self.number:
            index = _seq % 22 + 1
            # if index == 15 or index == 7 or index == 19:
            if index == 15 or index == 7 or index == 19 or index == 2 or index == 11 \
                    or index == 13 or index == 16 or index == 22:
                _seq += 1
                continue
            results.append(self.gen_one_query(index))
            actual += 1
            _seq += 1
        return results

    def gen_workloads2(self):
        results = list()
        query_type = list()
        actual = 0
        _seq = 0
        while actual < self.number:
            index = _seq % 22 + 1
            if index == 15 or index == 7 or index == 19:
                _seq += 1
                continue
            query_type.append(index)
            results.append(self.gen_one_query(index))
            actual += 1
            _seq += 1
        return results, query_type


In [55]:
w_size = 14
wd_generator = TPCH(work_dir, w_size)
workload = wd_generator.gen_workloads()



In [57]:
import numpy as np

def preprocess_workload(wd):
    
    with open("wl.txt",'w') as f:
        f.write(wd)
    data = ""
    with open('wl.txt', 'rt') as f:
        a = [x for x in f.readlines()]
    with open('wl.txt', 'rt') as f:
        i = 0
        for line in f.readlines():
            if 0 < i<len(a)-1:
                data = data + line
            i+=1

    return data

for i in range(len(workload)):
    workload[i] = preprocess_workload(workload[i])
with open('wl.pickle', 'wb') as df:
    pickle.dump(workload, df, protocol=0)


In [71]:
wf = open('data/workload.pickle', 'rb')
workload = pickle.load(wf)
wf.close()

In [65]:
wf1 = open('wl.pickle', 'rb')
workload = pickle.load(wf1)
wf1.close()

In [72]:
type(workload)
workload[5]

"-- using 1588346809 as a seed to the RNG\n\r\n\r\nselect\r\n\to_year,\r\n\tsum(case\r\n\t\twhen nation = 'KENYA' then volume\r\n\t\telse 0\r\n\tend) / sum(volume) as mkt_share\r\nfrom\r\n\t(\r\n\t\tselect\r\n\t\t\textract(year from o_orderdate) as o_year,\r\n\t\t\tl_extendedprice * (1 - l_discount) as volume,\r\n\t\t\tn2.n_name as nation\r\n\t\tfrom\r\n\t\t\tpart,\r\n\t\t\tsupplier,\r\n\t\t\tlineitem,\r\n\t\t\torders,\r\n\t\t\tcustomer,\r\n\t\t\tnation n1,\r\n\t\t\tnation n2,\r\n\t\t\tregion\r\n\t\twhere\r\n\t\t\tp_partkey = l_partkey\r\n\t\t\tand s_suppkey = l_suppkey\r\n\t\t\tand l_orderkey = o_orderkey\r\n\t\t\tand o_custkey = c_custkey\r\n\t\t\tand c_nationkey = n1.n_nationkey\r\n\t\t\tand n1.n_regionkey = r_regionkey\r\n\t\t\tand r_name = 'AFRICA'\r\n\t\t\tand s_nationkey = n2.n_nationkey\r\n\t\t\tand o_orderdate between date '1995-01-01' and date '1996-12-31'\r\n\t\t\tand p_type = 'LARGE BRUSHED COPPER'\r\n\t) as all_nations\r\ngroup by\r\n\to_year\r\norder by\r\n\to_year;\r\n"

In [11]:
import psqlparse
print(len(workload))
data = ""
with open('wl.txt', 'rt') as f:
    a = [x for x in f.readlines()]
with open('wl.txt', 'rt') as f:
    i = 0
    for line in f.readlines():
        if 0 < i<len(a)-1:
            data = data + " "+ line.strip()
        i+=1
print(data)

14
   select l_returnflag, l_linestatus, sum(l_quantity) as sum_qty, sum(l_extendedprice) as sum_base_price, sum(l_extendedprice * (1 - l_discount)) as sum_disc_price, sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) as sum_charge, avg(l_quantity) as avg_qty, avg(l_extendedprice) as avg_price, avg(l_discount) as avg_disc, count(*) as count_order from lineitem where l_shipdate <= date '1998-12-01' - interval '70' day (3) group by l_returnflag, l_linestatus order by l_returnflag, l_linestatus;


In [70]:
import psqlparse
# print(len(workload))
from Utility import Encoding as en
enc = en.encoding_schema()
parser = pi.Parser(enc['attr'])
print(parser.db_info.keys())
print(parser.index_candidates)
b= psqlparse.parse_dict(workload[5])# b= psqlparse.parse("select l_returnflag, sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) as sum_charge from lineitem where l_shipdata < 0 AND l_returnflag > 1 group by l_returnflag, l_linestatus;")
# print(len(b))
# print(b[0])
# print(parser.db_info["alias_attributes"])
parser.parse_stmt(b[0])

# print(b[0]['SelectStmt'].keys())
# where_clause = b[0]['SelectStmt']['whereClause']
# parser.parse_where_clause(where_clause)
parser.gain_candidates()
# print(parser.table_info["orders"].conditions)
added_i = set()
added_i.add('lineitem#l_shipmode')
added_i.add('lineitem#l_orderkey,l_shipmode')
added_i.add('lineitem#l_shipmode,l_orderkey')
print(parser.index_candidates)
print(parser.index_candidates | added_i)

dict_keys(['customer', 'lineitem', 'nation', 'orders', 'part', 'partsupp', 'region', 'supplier'])
set()
{'region#r_regionkey,r_name', 'lineitem#l_partkey', 'customer#c_custkey', 'region#r_name', 'region#r_name,r_regionkey', 'customer#c_nationkey', 'part#p_type', 'region#r_regionkey', 'supplier#s_suppkey', 'lineitem#l_suppkey', 'lineitem#l_orderkey', 'part#p_partkey', 'nation#n_nationkey', 'supplier#s_nationkey', 'orders#o_custkey', 'nation#n_regionkey', 'part#p_type,p_partkey', 'orders#o_orderkey', 'part#p_partkey,p_type'}
{'region#r_regionkey,r_name', 'lineitem#l_partkey', 'customer#c_custkey', 'region#r_name', 'region#r_name,r_regionkey', 'customer#c_nationkey', 'part#p_type', 'region#r_regionkey', 'supplier#s_suppkey', 'lineitem#l_orderkey,l_shipmode', 'lineitem#l_suppkey', 'lineitem#l_orderkey', 'part#p_partkey', 'nation#n_nationkey', 'lineitem#l_shipmode', 'supplier#s_nationkey', 'orders#o_custkey', 'nation#n_regionkey', 'part#p_type,p_partkey', 'lineitem#l_shipmode,l_orderkey', '

In [2]:
import psqlparse
# print(len(workload))
from Utility import ParserForIndex as pi
from Utility import Encoding as en
enc = en.encoding_schema()
parser = pi.Parser(enc['attr'])
parser.db_info.keys()

dict_keys(['customer', 'lineitem', 'nation', 'orders', 'part', 'partsupp', 'region', 'supplier'])

In [119]:
import pickle

df=open("cands2.pickle",'rb')#注意此处是rb
data3=pickle.load(df)
df.close()

data3

[]

In [182]:
import Utility.PostgreSQL as pg


pg_client = pg.PGHypo()
attributes = pg_client.get_attributes("lineitem", 'public')
print(attributes)
import psycopg2 as pg2
pg2.connect(database="tpch",user="wangtaiyi")

[]


<connection object at 0x7ff70f4f0ec0; dsn: 'user=wangtaiyi dbname=tpch', closed: 0>